In [10]:
import torch, time

In [2]:
A = torch.rand(10_000, 10_000) # default is float32, so this is about 400 MB

In [11]:
start = time.time()
A.sum(dim=0) # 0 is down, so this is summing over columns
end = time.time()
(end-start)*1000 # milliseconds

20.205020904541016

In [12]:
start = time.time()
A.sum(dim=1) # 0 is down, so this is summing over columns
end = time.time()
(end-start)*1000 # milliseconds

11.196613311767578

# PyArrow

In [4]:
# !wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.zip

In [6]:
# !unzip hdma-wi-2021.zip

In [7]:
!ls -lah

total 188M
drwxr-xr-x 3 ubuntu ubuntu 4.0K Dec 27 06:20 .
drwxr-xr-x 1 root   root   4.0K Dec 25 21:29 ..
drwxr-xr-x 2 root   root   4.0K Dec 27 06:14 .ipynb_checkpoints
-rw-r--r-- 1 root   root    22K Dec 27 06:13 cache.ipynb
-rw-r--r-- 1 root   root   117K Dec 26 01:55 demo.ipynb
-rw-r----- 1 root   root   167M Nov  1  2022 hdma-wi-2021.csv
-rw-r--r-- 1 root   root    21M Jan  5  2023 hdma-wi-2021.zip
-rw-r--r-- 1 root   root   3.4K Dec 27 06:20 pyarrow.ipynb
-rw-r--r-- 1 root   root    11K Dec 27 06:13 threads.ipynb


In [8]:
import pandas as pd

In [11]:
start = time.time()
pd.read_csv("hdma-wi-2021.csv")
end = time.time()
end-start # seconds

/tmp/ipykernel_3700/3879644429.py:2: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("hdma-wi-2021.csv")


13.11105728149414

In [2]:
import pyarrow.csv
import time

In [3]:
start = time.time()
tbl = pyarrow.csv.read_csv("hdma-wi-2021.csv")
end = time.time()
end-start # seconds

0.3916952610015869

In [5]:
start = time.time()
df = tbl.to_pandas()
end = time.time()
end-start # seconds

0.48350071907043457

In [6]:
import pyarrow.compute as pc

In [7]:
tbl["lei"].to_pandas()

0         54930034MNPILHP25H80
1         54930034MNPILHP25H80
2         54930034MNPILHP25H80
3         54930034MNPILHP25H80
4         54930034MNPILHP25H80
                  ...         
447362    54930034MNPILHP25H80
447363    54930034MNPILHP25H80
447364    54930034MNPILHP25H80
447365    54930034MNPILHP25H80
447366    54930034MNPILHP25H80
Name: lei, Length: 447367, dtype: object

In [8]:
pc.utf8_lower(tbl["lei"]).to_pandas()

0         54930034mnpilhp25h80
1         54930034mnpilhp25h80
2         54930034mnpilhp25h80
3         54930034mnpilhp25h80
4         54930034mnpilhp25h80
                  ...         
447362    54930034mnpilhp25h80
447363    54930034mnpilhp25h80
447364    54930034mnpilhp25h80
447365    54930034mnpilhp25h80
447366    54930034mnpilhp25h80
Length: 447367, dtype: object

In [9]:
tbl["income"]

[
  [
    108,
    103,
    146,
    70,
    71,
    ...
    null,
    null,
    null,
    null,
    null
  ],
  [
    null,
    null,
    null,
    null,
    null,
    ...
    null,
    null,
    null,
    null,
    null
  ],
...,
  [
    null,
    null,
    92,
    175,
    null,
    ...
    66,
    52,
    27,
    32,
    88
  ],
  [
    29,
    53,
    30,
    35,
    230,
    ...
    84,
    46,
    86,
    114,
    128
  ]
]

In [11]:
pc.mean(tbl["income"].drop_null()).as_py()

377.5220353645974

In [12]:
tbl[:10].to_pandas()

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2021,54930034MNPILHP25H80,99999,WI,55027,55027961800,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4196,3.67,69600,108,1422,1839,57
1,2021,54930034MNPILHP25H80,99999,WI,55001,55001950501,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,1511,5.43,69600,65,541,1966,33
2,2021,54930034MNPILHP25H80,99999,WI,55013,55013970400,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3895,9.63,69600,80,1685,5859,35
3,2021,54930034MNPILHP25H80,29404,WI,55059,55059002000,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5561,9.15,102500,106,1851,2208,30
4,2021,54930034MNPILHP25H80,11540,WI,55087,55087013300,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Joint,...,NaN,NaN,NaN,7248,5.22,85600,111,1939,2351,14
5,2021,54930034MNPILHP25H80,24580,WI,55061,55061960500,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6136,4.56,80100,104,2136,2627,53
6,2021,54930034MNPILHP25H80,43100,WI,55117,55117000100,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3803,16.49,79600,97,1011,1483,76
7,2021,54930034MNPILHP25H80,43100,WI,55117,55117000100,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3803,16.49,79600,97,1011,1483,76
8,2021,54930034MNPILHP25H80,99999,WI,55103,55103970100,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2909,3.71,69600,84,969,1490,67
9,2021,549300FQ2SN6TRRGB032,99999,WI,55127,55127001501,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,3160,15.06,69600,126,983,2219,22


# Day 2

In [3]:
import pyarrow as pa

In [4]:
batch = pa.RecordBatch.from_arrays([range(1,1_000_000),
                                    range(1,1_000_000),
                                    range(1,1_000_000)],
                                   names = ["x", "y", "z"])
                                    

In [5]:
batch.nbytes / 1024**2 # Megabytes

22.888160705566406

In [6]:
with pa.ipc.new_file("test.arrow", schema=batch.schema) as f:
    for i in range(50):
        f.write_batch(batch)

In [7]:
!ls -lah

total 1.4G
drwxr-xr-x 3 ubuntu ubuntu 4.0K Dec 27 06:50 .
drwxr-xr-x 1 root   root   4.0K Dec 27 06:46 ..
drwxr-xr-x 2 root   root   4.0K Dec 27 06:14 .ipynb_checkpoints
-rw-r--r-- 1 root   root    22K Dec 27 06:13 cache.ipynb
-rw-r--r-- 1 root   root   117K Dec 26 01:55 demo.ipynb
-rw-r----- 1 root   root   167M Nov  1  2022 hdma-wi-2021.csv
-rw-r--r-- 1 root   root    21M Jan  5  2023 hdma-wi-2021.zip
-rw-r--r-- 1 root   root    28K Dec 27 06:49 pyarrow.ipynb
-rw-r--r-- 1 root   root   1.2G Dec 27 06:50 test.arrow
-rw-r--r-- 1 root   root    11K Dec 27 06:13 threads.ipynb


In [ ]:
# with pa.ipc.open_file("test.arrow") as f:
#     tbl = f.read_all() # notebook crashed since our docker memory limit was 512 MB

In [9]:
import mmap
import pyarrow as pa

with open("test.arrow", "rb") as f:
    mm = mmap.mmap(f.fileno(), 0, access = mmap.ACCESS_READ)

In [10]:
mm[:10]

b'ARROW1\x00\x00\xff\xff'

In [12]:
with pa.ipc.open_file(mm) as f:
    tbl = f.read_all() # notebook did NOT crash since we are using mmap

In [13]:
import pyarrow.compute as pc


pc.sum(tbl["x"]

<pyarrow.Int64Scalar: 24999975000000>

In [15]:
pc.sum(tbl["y"])

<pyarrow.Int64Scalar: 24999975000000>

In [ ]:
pc.sum(tbl["z"])